В даному ноутбуці ми застосуємо методи регулярізації `L1` і `L2` до набору даних `Boston Housing` і порівняємо оцінку навчальної вибірки та оцінку тестової вибірки до та після використання цих методів.

In [10]:
from numpy import genfromtxt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split

In [11]:
dataset = genfromtxt("https://raw.githubusercontent.com/m-mehdi/tutorials/main/boston_housing.csv", delimiter=",")

In [12]:
dataset

array([[0.00000000e+00, 1.80000000e-01, 6.78152493e-02, ...,
        8.96799117e-02, 8.04248656e-03, 2.40000000e+01],
       [2.35922539e-04, 0.00000000e+00, 2.42302053e-01, ...,
        2.04470199e-01, 4.18080621e-02, 2.16000000e+01],
       [2.35697744e-04, 0.00000000e+00, 2.42302053e-01, ...,
        6.28144504e-02, 4.02790570e-03, 3.47000000e+01],
       ...,
       [6.11892474e-04, 0.00000000e+00, 4.20454545e-01, ...,
        1.07891832e-01, 1.16406475e-02, 2.39000000e+01],
       [1.16072990e-03, 0.00000000e+00, 4.20454545e-01, ...,
        1.29930407e-01, 1.71795127e-02, 2.20000000e+01],
       [4.61841693e-04, 0.00000000e+00, 4.20454545e-01, ...,
        1.69701987e-01, 2.87987643e-02, 1.19000000e+01]])

In [13]:
X = dataset[:,:-1]
y = dataset[:,-1]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Тепер ми можемо навчити модель лінійної регресії, а потім подивитися оцінку навчального набору та результат тестового набору:

In [15]:
l_r = LinearRegression().fit(X_train, y_train)

print(f"Linear Regression-Training set score: {l_r.score(X_train, y_train):.2f}")
print(f"Linear Regression-Test set score: {l_r.score(X_test, y_test):.2f}")

Linear Regression-Training set score: 0.94
Linear Regression-Test set score: 0.81


Порівняння продуктивності моделі на навчальному наборі та тестовому наборі показує, що модель страждає від оверфітингу.

Щоб уникнути оверфітингу та контролювати складність моделі, скористаємося `Ridge` регресією (регуляризація `L2`) і подивимося, наскільки добре це працює на наборі даних:

In [20]:
ridge = Ridge(alpha=0.7).fit(X_train, y_train)

print(f"Ridge Regression-Training set score: {ridge.score(X_train, y_train):.2f}")
print(f"Ridge Regression-Test set score: {ridge.score(X_test, y_test):.2f}")

Ridge Regression-Training set score: 0.88
Ridge Regression-Test set score: 0.84


Незважаючи на те, що оцінка навчального набору `Ridge` регресії трохи нижча, ніж оцінка навчальної лінійної регресії, оцінка тестового набору `Ridge` значно вища, ніж оцінка набору тесту лінійної регресії. Ці показники підтверджують, що `Ridge` регресія зменшує складність моделі.

Параметр альфа визначає компроміс між продуктивністю моделі на навчальному наборі та її простотою. Таким чином, збільшення значення альфа (його значення за замовчуванням `1.0`) спрощує модель за рахунок скорочення коефіцієнтів.

Тепер давайте застосуємо `LASSO`-регресію до набору даних і дослідимо результати.

In [35]:
lasso = Lasso(alpha=1.0).fit(X_train, y_train)

print(f"Lasso Regression-Training set score: {lasso.score(X_train, y_train):.2f}")
print(f"Lasso Regression-Test set score: {lasso.score(X_test, y_test):.2f}")

Lasso Regression-Training set score: 0.25
Lasso Regression-Test set score: 0.25


Як бачимо, `LASSO` працює досить невтішно, і це ознака андерфітингу. Модель `LASSO` не працює добре, тому що більшість коефіцієнтів стали рівними нулю. Якщо ми хочемо знати точну кількість функцій, які були використані в моделі, ми можемо це подивитися:

In [36]:
print(f"Number of features: {sum(lasso.coef_ != 0)}")

Number of features: 4


Це означає, що лише `4` із `104` ознак у навчальному наборі використовуються в регресійній моделі `LASSO`, а решта ігноруються.

Давайте відкоригуємо альфа, зменшивши його значення до `0.01`:

In [37]:
lasso = Lasso(alpha=0.01).fit(X_train, y_train)
print("Lasso Regression-Training set score: {:.2f}".format(lasso.score(X_train, y_train)))
print("Lasso Regression-Test set score: {:.2f}".format(lasso.score(X_test, y_test)))

Lasso Regression-Training set score: 0.88
Lasso Regression-Test set score: 0.82


/Users/oleh_komenchuk/Developer/HillelIT/venv/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.339e+00, tolerance: 3.510e+00
  model = cd_fast.enet_coordinate_descent(


In [38]:
print(f"Number of features: {sum(lasso.coef_ != 0)}")

Number of features: 30


Повторний запуск коду вище показує, що зі зменшенням альфа модель Lasso використовує `30` із `104` функцій.

Хоча ми можемо ще більше зменшити альфу, здається, що її оптимальне значення становить `0.01`.

Остання техніка, яку ми будемо використовувати, це `Elastic Net`.

In [50]:
elastic_net = ElasticNet(alpha=0.001, l1_ratio=0.7).fit(X_train, y_train)

print(f"Elastic Net-Training set score: {elastic_net.score(X_train, y_train):.2f}")
print(f"Elastic Net-Test set score: {elastic_net.score(X_test, y_test):.2f}")

Elastic Net-Training set score: 0.91
Elastic Net-Test set score: 0.85


/Users/oleh_komenchuk/Developer/HillelIT/venv/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.775e+03, tolerance: 3.510e+00
  model = cd_fast.enet_coordinate_descent(
